In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

True

In [ ]:
data=pd.read_csv('final_data_en.csv')
data=data[['text','class']]
data

,text,class
0,RT @maideeeni: I want to say a very big thank ...,anxiety
1,@AisYalcaE @clyrhck Done! I just need 320! Tha...,lonely
2,@SAD_istfied I'm gonna cry /pos,stress
3,RT @JoshOfficial___: I need a consistent fuck ...,anxiety
4,RT @mikegolicjr: glad we're finally talking ab...,normal
...,...,...
39761,Im nervous ash to fly though,anxiety
39762,64E7B6DA :Battle ID\nI need backup!\nLvl 120 G...,lonely
39763,SNOW MUCH FUN writing about our snow days in @...,anxiety
39764,Very time sensitive. I'm trans and my roommate...,anxiety


In [ ]:
import numpy as np
import tensorflow as tf
train=data.loc[ :33802]
test=data.loc[33803: ]
train=train.sample(frac=1)


train

,text,class
16945,@ChrisVanHollen Those senators that voted to t...,lonely
3408,RT @kyle_echarri: This isnt much but I hope th...,lonely
21899,THE DESCRIPTION OF THE DRESS IS SO BAD IM SORR...,lonely
32999,@NoLayingUp I know you talked to him a few yea...,lonely
23480,RT @Nandinibundini: Why Indians cucuk their bo...,anxiety
...,...,...
27363,RT @TeamJuJu: Happy to share that Ive donated ...,lonely
29677,I need ya to stop giving these people the atte...,lonely
10388,"1/ Out on my morning run in Shurugwi, Im remin...",anxiety
32149,RT @NHSActivistRN: TWEEPS I NEED YOUR HELP!\n\...,lonely


In [ ]:
train.dropna(inplace=True)



In [ ]:
train.isnull().sum()

text     0
class    0
dtype: int64

In [ ]:
train

,text,class
16945,@ChrisVanHollen Those senators that voted to t...,lonely
3408,RT @kyle_echarri: This isnt much but I hope th...,lonely
21899,THE DESCRIPTION OF THE DRESS IS SO BAD IM SORR...,lonely
32999,@NoLayingUp I know you talked to him a few yea...,lonely
23480,RT @Nandinibundini: Why Indians cucuk their bo...,anxiety
...,...,...
27363,RT @TeamJuJu: Happy to share that Ive donated ...,lonely
29677,I need ya to stop giving these people the atte...,lonely
10388,"1/ Out on my morning run in Shurugwi, Im remin...",anxiety
32149,RT @NHSActivistRN: TWEEPS I NEED YOUR HELP!\n\...,lonely


In [ ]:
import pandas as pd
import sklearn
import numpy as np
import re
import nltk

import matplotlib.pyplot as plt
from matplotlib import style

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as esw
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import string

# allow display of multiple outputs by running one code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
cachedStopWords = list(esw)
lemmatizer=WordNetLemmatizer()
#ps = PorterStemmer()

def lemmatize_article(sentence):
    sentence = word_tokenize(sentence)
    res = ''
    for word, tag in pos_tag(sentence):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        word = lemmatizer.lemmatize(word, wntag) if wntag else word
        res += word + ' '
    return res


def remove_stop_words(sentence):
    return ' '.join([word for word in sentence.split() if word not in cachedStopWords])
    
def remove_short(sentence):
    return ' '.join([word for word in sentence.split() if len(word) >= 3])
    
def remove_digits(sentence):
    return ' '.join([i for i in sentence.split() if not i.isdigit()])
    
def preprocess(all_texts):
    all_texts = list(map(lambda x: x.lower(), all_texts))
    all_texts = list(map(lambda x: x.translate(str.maketrans('', '', string.punctuation)), all_texts))
    all_texts = list(map(lambda x: lemmatize_article(x), all_texts))
    all_texts = list(map(lambda x: x.strip(), all_texts))
    all_texts = list(map(lambda x: remove_stop_words(x), all_texts))
    all_texts = list(map(lambda x: remove_short(x), all_texts))
    all_texts = list(map(lambda x: remove_digits(x), all_texts))
    return all_texts

In [ ]:
train['text'] = train['text'].str.replace(r'\s*@\w+', '', regex = True)
train['text'] = train['text'].str.replace(r'\s*\B@\w+', '', regex = True)
train['text'] = train['text'].str.replace(r'\s*@\S+', '', regex = True)
train['text'] = train['text'].str.replace(r'\s*@\S+\b', '', regex = True)
train['text'] = train['text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
train


,text,class
16945,Those senators that voted to table the impeac...,lonely
3408,RT: This isnt much but I hope this did leave a...,lonely
21899,THE DESCRIPTION OF THE DRESS IS SO BAD IM SORR...,lonely
32999,I know you talked to him a few years ago abou...,lonely
23480,RT: Why Indians cucuk their bodies lah? Not pa...,anxiety
...,...,...
27363,"RT: Happy to share that Ive donated $10,000 to...",lonely
29677,I need ya to stop giving these people the atte...,lonely
10388,"1/ Out on my morning run in Shurugwi, Im remin...",anxiety
32149,RT: TWEEPS I NEED YOUR HELP!\n\nThursday night...,lonely


In [ ]:
train['text'] = preprocess(train['text'])
train

,text,class
16945,senator vote table impeachment trial remove of...,lonely
3408,isnt hope leave beautiful memory 17th hop flow...,lonely
21899,description dress bad sorryi really need work,lonely
32999,know talk year ago topic need pod reason balan...,lonely
23480,indian cucuk body lah pain thaipusam like deep...,anxiety
...,...,...
27363,happy share ive donate provide medical assista...,lonely
29677,need stop people attention crave,lonely
10388,morning run shurugwi remind just committed tea...,anxiety
32149,tweeps need help thursday night campaign twitt...,lonely


In [ ]:
train = pd.DataFrame(train)
train.head()

,text,class
16945,senator vote table impeachment trial remove of...,lonely
3408,isnt hope leave beautiful memory 17th hop flow...,lonely
21899,description dress bad sorryi really need work,lonely
32999,know talk year ago topic need pod reason balan...,lonely
23480,indian cucuk body lah pain thaipusam like deep...,anxiety


In [ ]:
pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras as ks
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
X = train.text
Y =train['class']
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)


In [ ]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(300)(layer)
    layer = Dense(500,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(4,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='sparse_categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding_9 (Embedding)     (None, 150, 50)           50000     
                                                                 
 lstm_8 (LSTM)               (None, 300)               421200    
                                                                 
 FC1 (Dense)                 (None, 500)               150500    
                                                                 
 activation_8 (Activation)   (None, 500)               0         
                                                                 
 dropout_4 (Dropout)         (None, 500)               0         
                                                                 
 out_layer (Dense)           (None, 4)                 2004

In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
180/180 [==============================] - 310s 2s/step - loss: 0.9121 - accuracy: 0.6584 - val_loss: 0.5997 - val_accuracy: 0.7465
Epoch 2/10
180/180 [==============================] - 303s 2s/step - loss: 0.5579 - accuracy: 0.7730 - val_loss: 0.5837 - val_accuracy: 0.7647


In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)


159/159 [==============================] - 26s 166ms/step - loss: 0.5986 - accuracy: 0.7665


In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))


Test set
  Loss: 0.599
  Accuracy: 0.767
